In [1]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, InverseKinematics)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )
from pydrake.geometry import (MeshcatVisualizer, StartMeshcat)
from pydrake.visualization import (VisualizationConfig, ApplyVisualizationConfig, AddDefaultVisualization)

import numpy as np
from functools import partial

from collections import OrderedDict, namedtuple
ContactPairs = namedtuple("ContactPairs", "bodyA, bodyB, pts_in_A, pts_in_B, \
                           nhat_BA_W_list, miu")

conPairs = dict()
conPairs["h-obj"] = ContactPairs("panda_rightfinger", \
                                 "base_link_cracker", \
                                 [np.array([[0.1],[0.1],[0.1]])], \
                                 [np.array([[0.2],[0.2],[0.2]])], \
                                 [np.array([[0],[0],[1]])], \
                                 0.5)

conPairs["obj-env"] = ContactPairs("base_link_cracker", \
                                   "table_top_link", \
                                   [np.array([[0.1],[0.1],[0.1]])], \
                                   [np.array([[0.2],[0.2],[0.2]])], \
                                   [np.array([[1],[0],[1]])], \
                                   0.5)

"""
init the prog and setting decision variable
"""
# prog = MathematicalProgram()
# T = 2.
# N = 100
# q = prog.NewContinuousVariables(nq, N, "positions")
# v = prog.NewContinuousVariables(nv, N, "velocities")
# u = prog.NewContinuousVariables(nu, N, "joint_torques")
# lambdas = prog.NewContinuousVariables(nc*(nBasis+1+1), N, "contact_force_scalars") # 1 for fn, 1 for slack
# decVars = np.vstack((q,v,u,lambdas))


"""
Constraint functions
"""

# 1. (joint space + task space) dynamics 
# def generalized_dynamics_constraint(plant, contextList, decVars, contactPairs):
#     for ti in range(decVars.shape[1]):

"""
Some utility functions
"""
def AutoDiffArrayEqual(self, a, b):
    return np.array_equal(a, b) and np.array_equal(ExtractGradient(a), ExtractGradient(b))

class SceneFactory(object):
    def __init__(self, h, tableFile, pandaFile, boxFile, X_W_table, X_table_pandaBase, X_table_box, tipFile=None):
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=h)
        self.parser = Parser(self.plant)

        if tipFile == None:
            self.modelInstances= {"table_top" : self.parser.AddModels(tableFile)[0], \
                                   "panda"     : self.parser.AddModels(pandaFile)[0], \
                                   "box"       : self.parser.AddModels(boxFile)[0]}
        else:
            self.modelInstances = {"table_top" : self.parser.AddModels(tableFile)[0], \
                               "panda"     : self.parser.AddModels(pandaFile)[0], \
                               "box"       : self.parser.AddModels(boxFile)[0],
                               "tip"       : self.parser.AddModels(tipFile)[0]}

        self.table_top_frame = self.plant.GetFrameByName("table_top_center")
        self.robot_base_frame = self.plant.GetFrameByName("panda_link0")
        self.box_base_frame = self.plant.GetFrameByName("base_link_cracker")

        
        self.plant.WeldFrames(self.plant.world_frame(), self.table_top_frame, X_W_table)
        self.plant.WeldFrames(self.table_top_frame, self.robot_base_frame, X_table_pandaBase)
        self.plant.WeldFrames(self.table_top_frame, self.box_base_frame, X_table_box)

        self.plant.Finalize()
        # self.plant.set_name("Push3D")

        self.diagram = self.builder.Build()
        self.diagram_context = self.diagram.CreateDefaultContext()
        self.mutable_plant_context = self.plant.GetMyContextFromRoot(self.diagram_context)


"""
"A class for the case"
"""

class Push3D(object):
    def __init__(self, T, nStep, tableFile, \
                 pandaFile, boxFile, tipFile=None, \
                 X_W_table=None, X_table_pandaBase=None, X_table_box=None, visualize=False):
        self.builder = DiagramBuilder()
        self.tableFile = tableFile
        self.pandaFile = pandaFile
        self.boxFile = boxFile
        self.tipFile = tipFile
        self.isvisualized = visualize
        self.N = nStep
        self.h = T/nStep
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=self.h)
        self.parser = Parser(self.plant)

        if tipFile == None:
            self.modelInstances = {"table_top" : self.parser.AddModels(tableFile)[0], \
                                   "panda"     : self.parser.AddModels(pandaFile)[0], \
                                   "box"       : self.parser.AddModels(boxFile)[0]}
        else:
            self.modelInstances = {"table_top" : self.parser.AddModels(tableFile)[0], \
                               "panda"     : self.parser.AddModels(pandaFile)[0], \
                               "box"       : self.parser.AddModels(boxFile)[0],
                               "tip"       : self.parser.AddModels(tipFile)[0]}
        self.SetTheScene(X_W_table, X_table_pandaBase, X_table_box)

        self.builder_kin = DiagramBuilder()
        self.plant_kin, self.scene_graph_kin = AddMultibodyPlantSceneGraph(self.builder_kin, time_step=self.h)
        self.parser_kin = Parser(self.plant_kin)

        if tipFile == None:
            self.modelInstances_kin = {"table_top" : self.parser_kin.AddModels(tableFile)[0], \
                                   "panda"     : self.parser_kin.AddModels(pandaFile)[0], \
                                   "box"       : self.parser_kin.AddModels(boxFile)[0]}
        else:
            self.modelInstances_kin = {"table_top" : self.parser_kin.AddModels(tableFile)[0], \
                               "panda"     : self.parser_kin.AddModels(pandaFile)[0], \
                               "box"       : self.parser_kin.AddModels(boxFile)[0],
                               "tip"       : self.parser_kin.AddModels(tipFile)[0]}

    def SetTheScene(self, X_W_table, X_table_pandaBase, X_table_box):
        self.table_top_frame = self.plant.GetFrameByName("table_top_center")
        self.robot_base_frame = self.plant.GetFrameByName("panda_link0")
        self.box_base_frame = self.plant.GetFrameByName("base_link_cracker")

        if X_W_table != None:
            self.plant.WeldFrames(self.plant.world_frame(), self.table_top_frame, X_W_table)
        if X_table_pandaBase != None:
            self.plant.WeldFrames(self.table_top_frame, self.robot_base_frame, X_table_pandaBase)

        self.plant.Finalize()
        # self.plant.set_name("Push3D")

        # Two versions: float
        if self.isvisualized:
            self.meshcat = StartMeshcat()
            self.visualizer = MeshcatVisualizer.AddToBuilder(self.builder, self.scene_graph, self.meshcat)
            self.visualization_config = VisualizationConfig()
            self.visualization_config.publish_contacts = True
            self.visualization_config.publish_proximity = True
            ApplyVisualizationConfig(self.visualization_config, self.builder, meshcat=self.meshcat)
        self.diagram = self.builder.Build()
        self.diagram_context_list = [self.diagram.CreateDefaultContext() for i in range(self.N)]
        self.mutable_plant_context_list = [self.plant.GetMyContextFromRoot(context) for context in self.diagram_context_list]

        # set free body pose
        boxBody = self.plant.GetBodyByName("base_link_cracker")
        self.plant.SetFreeBodyPose(self.mutable_plant_context_list[0], boxBody, X_table_box)
        
        """ How to let ad diagram to co-exit with double diagram?? """
        # AutoDiff 
        # self.ad_diagram = self.diagram.ToAutoDiffXd()
        # self.ad_plant = self.ad_diagram.GetSubsystemByName("Push3D")
        # self.ad_mutable_context_list = [self.ad_plant.CreateDefaultContext() for i in range(self.N)]
    
    def CreateSceneForInvKinProg(self, X_W_table, X_table_pandaBase, X_table_box):
        """ create the same scene but weld the box as well, this is for testing the my own contacts 

        """
        return SceneFactory(self.h, self.tableFile, self.pandaFile, self.boxFile, X_W_table, X_table_pandaBase, X_table_box)
        
    def SetDevisionVariables(self, case, nContact, nBasis):
        self.prog = MathematicalProgram()
        self.nContactParams = nContact * (nBasis + 1 + 1)

        if case == "jointspace":
            # -2 is to offset the two fixed panda fingers
            self.nq = self.plant.num_positions(self.modelInstances["panda"]) - 2 + \
                 self.plant.num_positions(self.modelInstances["box"])
            self.nv = self.plant.num_velocities(self.modelInstances["panda"]) - 2 + \
                 self.plant.num_velocities(self.modelInstances["box"])
            self.nu = self.plant.num_actuators(self.modelInstances["panda"])

            self.qVars = self.prog.NewContinuousVariables(self.nq, self.N, "positions")
            self.vVars = self.prog.NewContinuousVariables(self.nv, self.N, "velocities")
            self.uVars = self.prog.NewContinuousVariables(self.nu, self.N, "actuations")
            self.contactVars = self.prog.NewContinuousVariables(self.nContactParams, self.N, "lambdas&slack")
            # 1 is for fn, 1 is for slack

        elif case == "taskspace":
            self.nq = self.plant.num_positions(self.modelInstances["tip"])  + \
                      self.plant.num_positions(self.modelInstances["box"])
            self.nv = self.plant.num_velocities(self.modelInstances["tip"])  + \
                      self.plant.num_velocities(self.modelInstances["box"])
            self.nu = self.plant.num_actuators(self.modelInstances["tip"])

            self.qVars = self.prog.NewContinuousVariables(self.nq, self.N, "positions")
            self.vVars = self.prog.NewContinuousVariables(self.nv, self.N, "velocities")
            self.uVars = self.prog.NewContinuousVariables(self.nu, self.N, "actuations")
            self.contactVars = self.prog.NewContinuousVariables(self.nContactParams, self.N, "lambdas&slack")

        self.nDecVar_ti = self.nq + self.nv + self.nu + nContact*(nBasis+1+1) # the offset for indexing
        self.nDecVar_all = (self.nq + self.nv + self.nu + nContact*(nBasis+1+1))*self.N

    def SetBoundaryConditions(self, decVars_t0, decVars_tf=None):
        """
        Sets the boundary conditions.
        
        :param      decVars_t0:  initial condition 
        :type       decVars_t0:  { 1D array}
        :param      decVars_tf:  final condition
        :type       decVars_tf:  { 1D array }
        """

        self.qVars[:, 0], self.vVars[:, 0], self.uVars[:, 0], self.contactVars[:, 0] = \
            np.split(decVars_t0, [self.nq, self.nq+self.nv, self.nq+self.nv+self.nContactParams])

        if decVars_tf != None:
            self.qVars[:, -1], self.vVars[:, -1], self.uVars[:, -1], self.contactVars[:, -1] = \
                np.split(decVars_tf, [self.nq, self.nq+self.nv, self.nq+self.nv+self.nContactParams])

    def GetnContact(self, conPairsDict):
        nContact = 0
        for key in conPairsDict:
            nContact += len(conPairsDict[key].pts_in_A)
        return nContact

    def SetSystemDynamicConstraint(self, conPairsDict):
        """
        "Note! The solver side expects a flattened vector, not 2D decVars"
        """
        # adding defect constraints 
        for i in range(self.N-1):
            self.prog.AddConstraint(partial(self.SystemDynamicConstraint, \
                                    context_index=i, conPairsDict=conPairsDict), \
                                    lb=[0]*self.nv, \
                                    ub=[0]*self.nv,\
                                    vars=np.concatenate(self.vVars[:, i], \
                                                        self.qVars[:, i+1],
                                                        self.vVars[:, i+1],
                                                        self.uVars[:, i+1],
                                                        self.contactVars[:, i+1]))

    def SetObjective(self):
        pass

    def Solve(self, decVar_init=None):
        if decVar_init != None:
            Solve(self.prog, initial_guess=initial_init)
        else:
            pass # zeros init

    def SystemDynamicConstraint(self, vars, context_index, conPairsDict, nBasis):
        nContact = self.GetnContact(conPairsDict)
        v, q_next, v_next, u_next, lambdasAndSlack_next = np.split(vars, \
                        [self.nv, self.nv+self.nq, self.nq+self.nv+self.nv, self.nq+self.nv+self.nv+self.nu])

        # split the contact vars for each contact
        markPoints = [(nBasis+1+1)*i for i in range(1, nContact)]
        lambdasAndSlack_set = [np.split(lambdasAndSlack_next, markPoints)]

        if isinstance(vars[0], AutoDiffXd):
            if not autoDiffArrayEqual(v, self.ad_plant.GetVelocities(self.mutable_context_list[context_index])):
                self.ad_plant.SetVelocities(self.mutable_context_list[context_index], v)
            if not autoDiffArrayEqual(q_next, self.ad_plant.GetPositions(self.mutable_context_list[context_index+1])):
                self.ad_plant.SetPositions(self.mutable_context_list[context_index+1], q_next)
            if not autoDiffArrayEqual(v_next, self.ad_plant.GetVelocities(self.mutable_context_list[context_index+1])):
                self.ad_plant.SetVelocities(self.mutable_context_list[context_index+1], v_next)

            B = self.ad_plant.MakeActuationMatrix()
            y = B @ u_next

            G_next = self.ad_plant.CalcGravityGeneralizedForces(self.mutable_context_list[context_index+1])
            y += G_next

            C_next = self.ad_plant.CalcBiasTerm(self.mutable_context_list[context_index+1])
            y -= C_next

            M_next = self.ad_plant.CalcMassMatrixViaInverseDynamics(self.mutable_context_list[context_index+1])

            # now loop to get contact Jacobians
            counter = 0
            for key, value in conPairDict:
                if key != "obj-env":
                    for i, (pt_in_A, pt_in_B) in zip(value.pts_in_A, value.pts_in_B):
                        Jv_V_W_Ca = self.ad_plant.CalcJacobianSpatialVelocity( \
                                                      self.mutable_context_list[context_index], 
                                                      JacobianWrtVariable.kV, 
                                                      self.ad_plant.GetFrameByName(value.bodyA), 
                                                      p_in_A, 
                                                      eval_plant.world_frame(), 
                                                      eval_plant.world_frame())
                        Jv_V_W_Cb = self.ad_plant.CalcJacobianSpatialVelocity( \
                                                      self.mutable_context_list[context_index], 
                                                      JacobianWrtVariable.kV, 
                                                      self.ad_plant.GetFrameByName(value.bodyB), 
                                                      p_in_B, 
                                                      eval_plant.world_frame(), 
                                                      eval_plant.world_frame())
                        v_tang_ACb_W = self.ComputeContactTangentialVelocity(context_index, \
                                                                             value.bodyA, \
                                                                             value.bodyB, \
                                                                             pt_in_B, \
                                                                             value.nhat_BA_W)

                        # need the offset the number of contact parameters each time
                        F_AB_W = self.ContactWrenchEvaluator(lambdasAndSlack_set[counter][:-1], \
                                                             value.nhat_BA_W, v_tang_ACb_W)
                        
                        y += Jv_V_W_Ca @ F_AB_W
                        y += Jv_V_W_Cb @ (-F_AB_W)
                        counter += 1
                    
                else:
                    for i, (pt_in_A, pt_in_B) in zip(value.pts_in_A, value.pts_in_B):
                        # only wrenches on the object is needed as table is fixed.
                        Jv_V_W_Ca = self.ad_plant.CalcJacobianSpatialVelocity( \
                                                      self.mutable_context_list[context_index], 
                                                      JacobianWrtVariable.kV, 
                                                      self.ad_plant.GetFrameByName(value.bodyA), 
                                                      p_in_A, 
                                                      eval_plant.world_frame(), 
                                                      eval_plant.world_frame())

                        v_tang_ACb_W = self.ComputeContactTangentialVelocity(context_index, \
                                                                             value.bodyA, \
                                                                             value.bodyB, \
                                                                             pt_in_B, \
                                                                             value.nhat_BA_W)
                        F_AB_W = self.ContactWrenchEvaluator(lambdasAndSlack_next[counter][:-1], \
                                                             value.nhat_BA_W, v_tang_ACb_W)

                        y += Jv_V_W_Ca @ F_AB_W # the wrench of A applied to B in expressed in {W}
                        counter += 1

            counter = 0
            y = y * self.h - M_next @ (v_next - v)
            
        else:
            # here goes a version for float datatype
            pass

    def ComputeConeBasis(self, v_tang_ACb_W):
        vhat_tang_ACb_W = v_tang_ACb_W/np.linalg.norm(v_tang_ACb_W)
        wrench = np.zeros((6, 1))
        angles = np.linspace(0, 360, lambdas.size-1) # exclude the lambdaN
        basis = []
        for i, ang in enumerate(angles):
            rot = np.array([[np.cos(ang), -np.sin(ang)],
                            [np.sin(ang),  np.cos(ang)]])
            basis.append(rot @ vhat_tang_ACb_W)
        return


    def ContactWrenchEvaluator(self, lambdas, nhat_BA_W, v_tang_ACb_W):
        # get the tangential velocity unit vector
        vhat_tang_ACb_W = v_tang_ACb_W/np.linalg.norm(v_tang_ACb_W)
        wrench = np.zeros((6, 1))
        angles = np.linspace(0, 360, lambdas.size-1) # exclude the lambdaN
        for i, ang in enumerate(angles):
            rot = np.array([[np.cos(ang), -np.sin(ang)],
                            [np.sin(ang),  np.cos(ang)]])
            wrench[3:] += (rot @ vhat_tang_ACb_W) * lambdas[i+1]

        wrench[3:] += lambdas[0] * nhat_BA_W  # contact normal
        return wrench

    def ComputeContactTangentialVelocity(self, context_index, bodyA, bodyB, ptB, nhat_BA_W):
        """
        "NOTE: ptB and nhat_BA_W are expected to be 1D vector, i,e (3,)"
        
        :param      context_index:  The context index
        :type       context_index:  { type_description }
        :param      bodyA:          The body a
        :type       bodyA:          { type_description }
        :param      bodyB:          The body b
        :type       bodyB:          { type_description }
        :param      ptB:            The point b
        :type       ptB:            { type_description }
        :param      nhat_BA_W:      The nhat ba w
        :type       nhat_BA_W:      { type_description }
        """
        frameA = self.ad_plant.GetFrameByName(bodyA)
        frameB = self.ad_plant.GetFrameByName(bodyB)
        V_AB_W = frameB.CalcSpatialVelocity(self.mutable_context_list[context_index], \
                                                    frameA, \
                                                    plant.world_frame())
        R_w_B = self.ad_plant.CalcRelativeTransform(self.mutable_context_list[context_index], \
                                                    plant.world_frame(), \
                                                    frameB).rotation() # DCM from body B pose in {W}
        p_BCb_W = R_w_B @ ptB # the contact points in {B} expressed in {W}
        v_ACb_W = V_AB_W.Shift(p_BCb_W).translational() # compute the contact point's velocity in {A}
        v_tang_ACb_W = v_ACb_W - np.dot(v_ACb_W, nhat_BA_W) * nhat_BA_W
        return v_tang_ACb_W

    def SceneVisualizer(self, context_index, jointPositions=None):
        # this is intended for visualization of the trajectory
        # as well as just visualize the scene at a time point
        if jointPositions is not None:
            plant_context_ti = self.mutable_plant_context_list[context_index]
            self.plant.SetPositions(plant_context_ti, self.modelInstances["panda"], jointPositions)
            self.plant.get_actuation_input_port().FixValue(plant_context_ti, np.zeros(9))
            self.diagram.ForcedPublish(self.diagram_context_list[context_index]) # publish the corresponding diagram

# Testing 1: testing the constructor of the Push3D( )

In [7]:
# import sys
# sys.path.append("/Users/xiao/0_codes/ICBM_drake")

# import dynamic_contact_problem as dcp
# import sliding_contact_constraint_LCP as scc

# initialize the scene
period = 2. # in seconds
N = 100
nContact = 5
nBasis = 6
case = "jointspace"

pandafile = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
boxfile = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/003_cracker_box.sdf"
tablefile =  "/Users/xiao/0_codes/ICBM_drake/models/objects/table_top.sdf"

X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])
X_table_pandaBase = RigidTransform(RotationMatrix.Identity(), [0.2, -0.4, 0.])
X_table_box = RigidTransform(RollPitchYaw(np.asarray([-90, 0, 0]) * np.pi / 180), [0.05, -0.25, 0.104])

traj = Push3D(period, N, tablefile, pandafile, boxfile, X_W_table=X_W_table, \
                  X_table_pandaBase=X_table_pandaBase, X_table_box=X_table_box, visualize=True)
traj.SetDevisionVariables(case, nContact, nBasis)

# testing decision variable dimensions
if traj.nq + traj.nv + traj.nu + nContact * (nBasis+1+1) == traj.nDecVar_ti:
	print("The dim of decision variables at ti matches! \n")
if (traj.nq + traj.nv + traj.nu + nContact * (nBasis+1+1)) * N == traj.nDecVar_all:
	print("The dim of total decision variables matches! \n")

# visualize the case in meshcat
context_index = 0
jointPositions = np.array([0.42008407, -1.29134329,  1.71323444, -2.50139919,  2.10728812,
        2.51729492,  0.7150981 ,  0.        ,  0.  ])

traj.SceneVisualizer(context_index, jointPositions)

# using the pydrake.geometry.QueryObject.inspector() to fetch the contact point
query_port = traj.plant.get_geometry_query_input_port()
if query_port.HasValue(traj.mutable_plant_context_list[context_index]):
    query_object = query_port.Eval(traj.mutable_plant_context_list[context_index])

# Next up, to get the contact in current scene
# referring to link here: 
# https://stackoverflow.com/questions/71590740/is-there-a-way-to-get-collision-results-in-the-context-of-the-plant

INFO:drake:Meshcat listening for connections at http://localhost:7002


The dim of decision variables at ti matches! 

The dim of total decision variables matches! 



In [12]:
# get the pose of the box
box_frame = traj.plant.GetFrameByName("base_link_cracker")
box_pose_w = box_frame.CalcPoseInWorld(traj.mutable_plant_context_list[context_index])

X_box_pt = RigidTransform(
  R=RotationMatrix(),
  p=[0., -0.082, 0.1],
) 

# what I want is to let this point meet a point in the hand expressed in link8
point_on_box_w = X_box_pt.multiply(box_pose_w)

# get finger pose in link 8
finger_frame = traj.plant.GetFrameByName("panda_rightfinger")
link8_frame = traj.plant.GetFrameByName("panda_link8")
fingerPose_desired_w = RigidTransform(RollPitchYaw(np.asarray([-135, 0, 0]) * np.pi / 180), [0., 0., 0.])
box_pose_w

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.05, -0.25, 0.104],
)

In [13]:
# a point on the right finger in its frame
ptFinger_b = [-0.01, 0.01, 0.01]
invTest = traj.CreateSceneForInvKinProg(X_W_table, X_table_pandaBase, X_table_box)

In [15]:
ik = InverseKinematics(invTest.plant, invTest.mutable_plant_context)
ik.AddPositionConstraint(
        finger_frame,
        ptFinger_b,
        invTest.plant.world_frame(),
        point_on_box_w.translation(),
        point_on_box_w.translation(),
    )
ik.AddOrientationConstraint(
        link8_frame,
        RotationMatrix(),
        invTest.plant.world_frame(),
        fingerPose_desired_w.rotation(),
        0.,
    )
prog = ik.get_mutable_prog()
q = ik.q()
print(q)
# q0 = invTest.plant.GetPositions(invTest.mutable_plant_context, invTest.modelInstances["panda"])
# q0 = np.zeros(9)
q0 = [0.09968388, -1.41256794,  1.77917944, -2.13052544,  1.54486971, 2.2048219,  0.34531915,  0.        ,  0.]
prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
prog.SetInitialGuess(q, q0)
result = Solve(ik.prog())
if result.is_success():
    print("IK success")
else:
    print("IK failure")
result.GetSolution()

[Variable('q(0)', Continuous) Variable('q(1)', Continuous)
 Variable('q(2)', Continuous) Variable('q(3)', Continuous)
 Variable('q(4)', Continuous) Variable('q(5)', Continuous)
 Variable('q(6)', Continuous) Variable('q(7)', Continuous)
 Variable('q(8)', Continuous)]
IK success


array([ 0.42008407, -1.29134329,  1.71323444, -2.50139919,  2.10728812,
        2.51729492,  0.7150981 ,  0.        ,  0.        ])

# Solving IK as an constrained optimization (using FKin)
Try example 6.2 here to better understand adding constraints: https://manipulation.csail.mit.edu/trajectories.html

In [19]:
def InvKinSpaceProg(plant, mutable_context, frame4trans, frame4rot, trans_w, rot_w, q0, 
                    rot_tol=0., trans_offset=[0., 0., 0.], rot_offset=RotationMatrix()):
    """
    Compute the inverse kinematics given a pose with two part. This may handy in 
    cases like mine: 
    The distance of two points constraint lie within the bounding box, can be fixed to 
        set dis = 0, i.e. .AddPositionConstraint()
    The difference in orientations of two poses (in their corresponding frames) can be set to
        0 for perfect alignment, i.e. .AddOrientationConstraint()

    In my case I use link_8 for the orientation constraint because I know the frame of 
    link8 to fiddle with, I don't know the finger frame. 
    
    """
    ik = InverseKinematics(plant, mutable_context)
    ik.AddPositionConstraint(
        frame4trans,
        trans_offset,
        plant.world_frame(),
        trans_w,
        trans_w)
    
    ik.AddOrientationConstraint(
        frame4rot,
        rot_offset,
        plant.world_frame(),
        rot_w,
        rot_tol)
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if result.is_success():
        print("IK success")
    else:
        print("IK failure")
    return result.GetSolution()


jointPos = InvKinSpaceProg(invTest.plant, 
                           invTest.mutable_plant_context, 
                           finger_frame, 
                           link8_frame, 
                           point_on_box_w.translation(), 
                           fingerPose_desired_w.rotation(), 
                           q0, 
                           rot_tol=0., trans_offset=ptFinger_b, rot_offset=RotationMatrix())
print(jointPos)

IK success
[ 0.42008407 -1.29134329  1.71323444 -2.50139919  2.10728812  2.51729492
  0.7150981   0.          0.        ]


# Solving IK using pseudo-inverse (Newton-Raphson)

In [175]:
# desired twist
rightFinger_frame = invTest.plant.GetFrameByName("panda_rightfinger")
V_d_w = rightFinger_frame.CalcSpatialVelocity(invTest.mutable_plant_context, invTest.box_base_frame, invTest.box_base_frame)
V_d_b = rightFinger_frame.CalcSpatialVelocity(invTest.mutable_plant_context, invTest.box_base_frame, invTest.plant.world_frame())

In [176]:
V_d_w

SpatialVelocity(
  w=[0.0, 0.0, 0.0],
  v=[0.0, 0.0, 0.0],
)

In [177]:
V_d_b

SpatialVelocity(
  w=[0.0, 0.0, 0.0],
  v=[0.0, 0.0, 0.0],
)

# Solving IK using pseudo-inverse as an optimization